In [1]:
import pandas as pd
from os.path import join, expanduser
from glob import glob
import xml.etree.ElementTree as ET

In [2]:
home = expanduser('~')

demog_csv_path = join(home, 'Downloads', 'PTDEMOG.csv')
df = pd.read_csv(demog_csv_path)
df['Category'] = None

In [3]:
df_phases = pd.unique(df['Phase']).tolist()

In [4]:
def find_xml(xml_file_list, subject, identifier):
    for xml_file_path in xml_file_list:
        xml_file = xml_file_path.split('/')[-1]
        if subject in xml_file and identifier in xml_file:
            return xml_file_path

    return None

In [5]:
dataset_path = join(home, 'Work', 'ADNI_Project', 'Data_revision')
categories = ['AD', 'CN', 'MCI']
tracers = ['AV45', 'PiB']

patients_searched = []
select_indices = []

for category in categories:
    for tracer in tracers:
        scan_dirs = glob(join(dataset_path, category, tracer, '*'))
        scan_dirs = list(filter(lambda path: 'Metadata' not in path, scan_dirs))
        scan_dirs = list(filter(lambda path: 'stats' not in path, scan_dirs))

        metadata_dir = join(dataset_path, category, tracer, 'Metadata', 'ADNI')
        metadata_files = glob(join(metadata_dir, '*.xml'))

        phase = ''

        # print(scan_dirs[:5])
        # continue
        for scan_dir in scan_dirs:
            scan_name = scan_dir.split('/')[-1]
            subject = scan_name.split('~')[0]
            identifier = scan_name.split('~')[-1]

            rid = int(subject.split('_')[-1])
            metadata_xml = find_xml(metadata_files, subject, identifier)

            root = ET.parse(metadata_xml).getroot()
            phase_xml = root.findall(".//visitIdentifier")[0].text

            for df_phase in df_phases:
                if df_phase in phase_xml:
                    phase = df_phase
            try:
                df.at[df.index[(df['Phase'] == phase) & (df['RID'] == rid)], 'Category'] = category
                select_indices.append(df.index[(df['Phase'] == phase) & (df['RID'] == rid)].tolist()[0])
            except IndexError:
                pass
                # print('Scan: ', scan_dir, '\tPhase: ', phase, '\tRID: ', rid)
# select_indices

In [6]:
sub_df = df.iloc[select_indices]
sub_df.to_csv('demographic.csv', index=False)

In [7]:
sub_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 531 entries, 3258 to 510
Data columns (total 33 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Phase         531 non-null    object 
 1   ID            531 non-null    int64  
 2   RID           531 non-null    int64  
 3   SITEID        531 non-null    int64  
 4   VISCODE       531 non-null    object 
 5   VISCODE2      531 non-null    object 
 6   USERDATE      531 non-null    object 
 7   USERDATE2     150 non-null    object 
 8   PTSOURCE      531 non-null    int64  
 9   PTGENDER      531 non-null    float64
 10  PTDOBMM       531 non-null    float64
 11  PTDOBYY       531 non-null    float64
 12  PTHAND        531 non-null    float64
 13  PTMARRY       531 non-null    float64
 14  PTEDUCAT      531 non-null    float64
 15  PTWORKHS      180 non-null    float64
 16  PTWORK        531 non-null    object 
 17  PTWRECNT      531 non-null    object 
 18  PTNOTRT       531 non-null 